In [1]:
import torch
import glob
import os
# glob 결과 숫자 오름차순으로 정리해주는 라이브러리, 기능적으로 필요하지 않았음을 깨달았으나 
# 정렬 작업이 유지보수를 가정했을 때 충분히 의미 있다고 생각해서 그냥 놔두기로 함
import natsort 
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from torchvision import models
from torchsummary import summary

import numpy as np

In [2]:
# transform을 적용한 커스텀 데이터셋
# 무조건 torch.utils.data.Dataset을 상속받아야 한다.
class cnd_data(torch.utils.data.Dataset):
    def __init__(self, file_path, train=True, transforms=None):

        self.train=train
        self.transforms=transforms

        # cat, dog 경로 설정
        self.cat_img_path=os.path.join(file_path, 'data\kagglecatsanddogs\PetImages\Cat')
        self.dog_img_path=os.path.join(file_path, 'data\kagglecatsanddogs\PetImages\Dog')
        
        # cat, dog 이미지 목록 불러오기
        self.cat_list=natsort.natsorted(glob.glob(self.cat_img_path + '/*.jpg'))
        self.dog_list=natsort.natsorted(glob.glob(self.dog_img_path + '/*.jpg'))
 
        # cat, dog 이미지 list 및 label 지정하기, 0은 cat이고, 1은 dog이다
        # cat, dog 각각 12500개의 이미지가 존재하며, 각각 12000개는 train, 500개는 test에 사용된다
        if self.train==True:
            self.imgn_list=self.cat_list[:12000]+self.dog_list[:12000]
            self.img_label=[0]*12000+[1]*12000

        else:
            self.imgn_list=self.cat_list[12000:]+self.dog_list[12000:]
            self.img_label=[0]*500+[1]*500

        # 한번에 모든 이미지를 메모리에 올리고 싶었지만 공간 부족으로 불가
        # getitem쪽에 올렸다.


    # __len__()은 데이터쌍의 개수를 의미한다.
    # 아마 __len__의 크기를 기준으로 Dataloader에서 batch 묶음의 수를 결정하고 
    # __len__만큼의 데이터쌍을 가져오는 것 같다.
    def __len__(self):
        return len(self.img_label)

    # __getitem__()은 하나의 데이터쌍(보통 데이터, 레이블)을 가져오는데 사용된다.
    # __getitem__출력시 한 쌍의 데이터가 아니라 한 batch만큼을 한번에 불러오는 방식으로 짜고 싶었지만
    # (만약 그렇게 한다면 Dataloader에서 불러온 다음 중첩 for문을 사용하여 사용하게 될 것이다.)
    # 처음으로 짜는 커스텀 데이터셋이기 때문에 한 쌍의 데이터를 가져올 때마다
    def __getitem__(self, idx):

        # 원 데이터는 cat과 dog 폴더로 나뉘어 있으며, 각각 0~12499까지 숫자가 파일 이름으로 사용된다.
        # 또한, train은 0~11999, test는 12000~12499 를 파일 이름으로 사용한다.
        # train 기준 실존하는 imgn_list의 index는 0~23999까지 사용하게 되므로, 
        # 0~11999 idx의 경우 cat폴더에서 가져와야 하며,
        # 12000~23999 idx의 경우 dog 폴더에서 가져와야 한다.(당연히 dog폴더의 train이미지는 0~11999이므로 숫자 변환도 필요하다)
        # 라고 처음에는 생각해 왔지만 헛생각이었다... 어차피 인덱스와 이에 해당하는 이미지 경로는 연결되어 있으니 추가적인 조치를 취하지 않고도
        # 문제를 해결할 수 있다. 
        image_data=Image.open(self.imgn_list[idx]).convert('RGB')

                
        # if len(np.array(image_data).shape)==2:
        #     image_data=image_data.convert('RGB')
        #     print('변환 후 사이즈:',np.array(image_data).shape)
        
        if self.transforms:
            sample=self.transforms(image_data)

        # print('사이즈:', sample.size())
        # 이미지에서 channel이 3이 아닌 경우
        # if sample.size()[0] != 3:
        #     print(self.imgn_list[idx])
        #     print('변환 사이즈:', sample.size())
            
            # sample=sample.expand(3, -1, -1)
            # print(sample.size())

        return sample, self.img_label[idx]

In [3]:
# 경로 설정, py파일로 변환시 경로는 변경되어야 한다.
path=os.path.abspath('../')

# Resize: 크기를 224, 224로 맞춘다
# ToTensor: 데이터 타입을 Tensor로 만든다. Tensor의 원소는 0~1로 정해진다.(https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor)
# custom으로 transform를 작성하는 것도 가능하다.
transforms=transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()])

In [4]:
cnd_train=cnd_data(file_path=path, train=True, transforms=transforms)

In [5]:
cnd_dataloader=DataLoader(cnd_train, batch_size=32, shuffle=True)

In [6]:
# resnet34나 50이나 조금씩 원문에서 주장하는 model을 수정한 듯한 흔적이 보인다.
# 하지만 지금은 resnet 원문의 것을 구현하는 입장이기 때문에
# 모델 참조를 하다 원문과 다른 부분이 있다면 무시하고 원문대로 한다.
resnet34_preset=models.resnet34()
summary(resnet34_preset, input_size=(3, 224, 224))
print(resnet34_preset)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [7]:
resnet50_preset=models.resnet50()
summary(resnet50_preset, input_size=(3, 224, 224))
print(resnet50_preset)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [8]:
# 모델을 정의할 때는 무조건 torch.nn.Module을 상속받아야 한다.
# block은 short connection이 있는 최소 단위이며 
# group은 동일한 block 형성 패턴(논문 table 1의 conv2_x, conv3_x)을 의미한다.
class ResNet_compat(nn.Module):
    def __init__(self,
                 input_shape=(3, 224, 224),
                 blocks_in_model=[3, 4, 6, 3],
                 layers_in_block=[2, 2, 2, 2],
                 kernel_sizes=[(3,3), (3,3), (3,3), (3,3)],
                 channel_sizes=[(64,64), (128,128), (256,256), (512,512)],
                 class_size=2,
                 is_plain=False):

        super(ResNet_compat, self).__init__()

        self.input_shape=input_shape
        self.blocks_in_model=blocks_in_model
        self.layers_in_block=layers_in_block
        self.kernel_sizes=kernel_sizes
        self.channel_sizes=channel_sizes
        self.class_size=class_size
        self.is_plain=is_plain


        # pytorch에도 padding='same'이라는 옵션은 존재하지만, stride=1일
        # 경우만 사용 가능하다.
        # 아래 코드는 (W-F+2P)/S + 1 공식 적용한 코드로
        # 계산 결과가 소수점이 나오지만, pytorch에서 사용하는 resnet이 이렇게 설정하였기 때문에
        # 똑같이 진행한다. 
        
        # conv1+conv2 maxpooling
        self.conv1=nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(), # 왜 그런지 모르겠지만 preset에는 inplace=True(default: False)
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)
        )

        # conv2(maxpooling은 제외), short connection은 구현부에서 구현
        
        self.block_forms=list()
        for i in range(len(blocks_in_model)):
            for j in range(blocks_in_model[i]):
                
                # i는 group, j는 block
                if i==0 and j==0:
                    inp_channel=64
                elif i!=0 and j==0:
                    inp_channel=channel_sizes[i-1][-1]
                else:
                    inp_channel=False

                # # input_channel은 block의 맨 처음+group(group: block묶음)의 맨 처음인 경우만 사용됨
                # self.block_forms.append(nn.Sequential(*self.build_block(
                #     self.layers_in_block[i],
                #     kernel_sizes=self.kernel_sizes[i],
                #     channel_sizes=self.channel_sizes[i],
                #     input_channel=inp_channel,
                #     is_plain=False
                # )))
                # input_channel은 block의 맨 처음+group(group: block묶음)의 맨 처음인 경우만 사용됨
                self.block_forms.append(self.build_block(
                    self.layers_in_block[i],
                    kernel_sizes=self.kernel_sizes[i],
                    channel_sizes=self.channel_sizes[i],
                    input_channel=inp_channel,
                    is_plain=False
                ))



        # summary가 안 먹혀서 새로 짠 코드
        # self.model_body=nn.Sequential(*self.block_forms)
        
        # 선언을 하려고 하면 추가 입력이 필요해서 forward에 설정해야 하는 상황...
        self.relu = nn.ReLU()

        self.end_avg2d=nn.AdaptiveAvgPool2d((1,1))
        self.end_linear=nn.Linear(in_features=self.channel_sizes[-1][-1], out_features=2, bias=True)
        # self.end_linear=nn.Linear(in_features=512*int(channel_sizes[-1][0]/channel_sizes[-1][-1]), out_features=2, bias=True)
        # self.end_linear=nn.Linear(in_features=512, out_features=2, bias=True)
        self.end_softmax=nn.Softmax(-1)
    
    # input_channel은 block의 맨 처음+group(group: block묶음)의 맨 처음인 경우만 사용됨
    def build_block(self, layers, kernel_sizes, channel_sizes, input_channel, is_plain=False):
        
        full_block=[]
        for i in range(layers):
            if kernel_sizes[i]!= 1:
                layer_padding=(1,1)
            else:
                layer_padding=(0,0)

            if input_channel and i==0:
                if input_channel != channel_sizes[i]:
                    f_stride=2
                else:
                    f_stride=1
                full_block.append(nn.Conv2d(in_channels=input_channel, 
                                            out_channels=channel_sizes[i], 
                                            kernel_size=kernel_sizes[i],
                                            padding=layer_padding,
                                            stride=f_stride,
                                            bias=False
                                            ))
            else:
                # 50이상은 channel이 일시적으로 늘어나도 feature map의 크기가 그대로임음 명심할 것
                # padding만 어떻게 할지 고민해보자... kernel_size가 1일 때는 패딩 제외?? 아니면 3일 때만 padding 1??
                full_block.append(nn.Conv2d(in_channels=channel_sizes[i-1],
                                            out_channels=channel_sizes[i],
                                            kernel_size=kernel_sizes[i],
                                            padding=layer_padding,
                                            bias=False
                                            ))
            # batch_normalization
            full_block.append(nn.BatchNorm2d(channel_sizes[i], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))

            # short connection은 구현부에서 만들기 때문에 block의 마지막 layer가 아니라면 relu 추가
            # inplace 옵션이 있고, pre-model에서는 사용하긴 사용하지만 왜 사용하는지 모르겠어서 사용 안함
            if i< layers-1:
                full_block.append(nn.ReLU())

        return nn.Sequential(*full_block)
        
    def forward(self, x):
        # print(x.shape)
        x=self.conv1(x)
        # print(x.shape)

        # model body
        for block in self.block_forms:
            identity=x
            x=block(x)

            if block[0].in_channels != block[0].out_channels:
                self.reduce=nn.Conv2d(
                    block[0].in_channels,
                    block[0].out_channels,
                    kernel_size=(1,1),
                    stride=2)
                identity=self.reduce(identity)
            
            x+=identity
            x=self.relu(x)

        
        # 끝단
        x=self.end_avg2d(x)
        x=torch.flatten(x, 1, -1)

        # x현재 shape는 [2, 512, 1, 1]
        x=self.end_linear(x)
        x=self.end_softmax(x)
       
        return x

        # return self.conv_temp(x)
        
        pass

In [9]:
train_model=ResNet_compat()
summary(train_model, input_size=(3, 224, 224))
print(train_model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
              ReLU-5           [-1, 64, 56, 56]               0
              ReLU-6           [-1, 64, 56, 56]               0
              ReLU-7           [-1, 64, 56, 56]               0
              ReLU-8          [-1, 128, 28, 28]               0
              ReLU-9          [-1, 128, 28, 28]               0
             ReLU-10          [-1, 128, 28, 28]               0
             ReLU-11          [-1, 128, 28, 28]               0
             ReLU-12          [-1, 256, 14, 14]               0
             ReLU-13          [-1, 256, 14, 14]               0
             ReLU-14          [-1, 256,

In [10]:
learning_rate=0.01

loss_f= nn.CrossEntropyLoss()
# train_model.parameters: 최적화할 대상의 파라미터
# lr=learning_rate
optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate)

In [11]:
# dogs 11285, 8730, 11675 3588, 5604(not dog), 11853, 2877, 6318, 9078(channel 4), 11410 /3588와 5604가 중복해서 나옴. 특정 데이터 문제일 가능성이 높아짐
# cats 8470, 5686, 9778, 2877, 7276, 11935, 5370
EPOCHS=30

for epoch in range(EPOCHS):
    losses=[]
    # running loss 해당 단계에서의 loss 누적값
    running_loss=0

    for i, inp in enumerate(cnd_dataloader):

        inputs, labels= inp
        # 모든 gradient를 0으로 설정, 이렇게 하지 않으면 이전 loop의 gradient값이 그대로 남아있어 제대로 학습이 되지 않는다.
        optimizer.zero_grad()

        # train_model을 태운 다음 loss를 계산한다.
        outputs= train_model(inputs)
        loss= loss_f(outputs, labels)
        # 계산한 loss를 losses에 추가한다.
        losses.append(loss.item())

        # loss.backward()로 gradient를 계산하고
        # optimizer를 사용하여 반영한다.
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

        if i%5 == 0 and i>0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss/100)
            running_loss=0.0

    avg_loss= sum(losses)/len(losses)


Loss [1, 5](epoch, minibatch):  0.042775524258613584
Loss [1, 10](epoch, minibatch):  0.03686631858348846
Loss [1, 15](epoch, minibatch):  0.03535716474056244
Loss [1, 20](epoch, minibatch):  0.035277345180511475
Loss [1, 25](epoch, minibatch):  0.03496389806270599
Loss [1, 30](epoch, minibatch):  0.035166540145874024
Loss [1, 35](epoch, minibatch):  0.034430102705955506
Loss [1, 40](epoch, minibatch):  0.0345890748500824
Loss [1, 45](epoch, minibatch):  0.03416333973407745
Loss [1, 50](epoch, minibatch):  0.034757272601127626
Loss [1, 55](epoch, minibatch):  0.03518288850784302
Loss [1, 60](epoch, minibatch):  0.03542967915534973
Loss [1, 65](epoch, minibatch):  0.034873214960098264
Loss [1, 70](epoch, minibatch):  0.03426034390926361
Loss [1, 75](epoch, minibatch):  0.03476904332637787
Loss [1, 80](epoch, minibatch):  0.033925381302833554
Loss [1, 85](epoch, minibatch):  0.03570381224155426
Loss [1, 90](epoch, minibatch):  0.03537484288215637
Loss [1, 95](epoch, minibatch):  0.033929

KeyboardInterrupt: 